# Instagram Deep Scraping Cleaning
We have use the library `instaloader` to scrap the instagram posts and comments since 2019 for Offcorss, EPK and Politokids profiles. Each post is stored in a separated `.json` file, and the ones that have any comment also have a separated file for the list of comments.

In this notebook we are going to read all those files for each company and get them into a new file that follows the schema of the database that we have created.

In [1]:
import pandas as pd
import numpy as np
import json
import os
import glob
import preprocessor as p
from datetime import datetime

----------------------------

## Reading all files in a Pandas DF

First, we are going to iter in the folders of the profiles names. For each file we are going to open it in a dataframe, and store it in a list deppending if it is a post or a comment. At the end, we are going to append all the openned dataframes to a `ig_posts` and `ig_comments` dataframes.

In [2]:
company_names = ['offcorss', 'epk', 'politokids']

company_posts_df = []
company_comments_df = []

for company in company_names:
    files_list = glob.glob(f'{company}/*.json')
    post_list = [file for file in files_list if not 'comments' in file]
    
    df_posts_list = []
    for post in post_list:
        temp_post_df = pd.read_json(post).drop(columns='instaloader')
        df_posts_list.append(pd.read_json(post).drop(columns='instaloader'))
        comment = post.replace('.json', '_comments.json')
        if comment in files_list:
            temp_comment_df = pd.read_json(comment)
            temp_comment_df['post_id'] = temp_post_df.loc['id','node']
            temp_comment_df['post_shortcode'] = temp_post_df.loc['shortcode','node']
            temp_comment_df['brand'] = company
            company_comments_df.append(temp_comment_df)
    
    company_posts = pd.concat(df_posts_list, axis=1).transpose()
    df_info = ['id', 'shortcode', 'edge_media_to_caption', 'taken_at_timestamp', '__typename', 'is_ad', 'is_video', 'edge_media_preview_like', 'edge_media_to_comment']
    company_posts = company_posts[df_info]
    company_posts['posts_company'] = company
    company_posts_df.append(company_posts)
    
ig_posts = pd.concat(company_posts_df)
ig_comments = pd.concat(company_comments_df)

In [3]:
copyp = ig_posts.copy()
copyc = ig_comments.copy()

In [4]:
ig_posts.head()

,id,shortcode,edge_media_to_caption,taken_at_timestamp,__typename,is_ad,is_video,edge_media_preview_like,edge_media_to_comment,posts_company
node,1946502669929734835,BsDXd2DguKz,{'edges': [{'node': {'text': '¡FELIZ AÑO NUEVO...,1546261222,GraphImage,NaN,False,{'count': 308},{'count': 0},offcorss
node,1946624133611601575,BsDzFX7Hz6n,{'edges': [{'node': {'text': '¡Hoy es un día p...,1546275701,GraphSidecar,NaN,False,{'count': 212},{'count': 0},offcorss
node,1946744414957729955,BsEObsqB_ij,{'edges': [{'node': {'text': '¡Estamos listos ...,1546290040,GraphImage,NaN,False,{'count': 278},{'count': 0},offcorss
node,1947257691818809835,BsGDI2_Adnr,{'edges': [{'node': {'text': 'Día 1 del año 🎉⭐...,1546351227,GraphImage,False,False,"{'count': 126, 'edges': []}","{'count': 1, 'edges': [], 'page_info': {'end_c...",offcorss
node,1947346707928591260,BsGXYNsnCuc,{'edges': [{'node': {'text': 'UN NUEVO AÑO com...,1546361839,GraphSidecar,NaN,False,{'count': 391},{'count': 0},offcorss


In [5]:
ig_comments.head()

,id,created_at,text,owner,likes_count,answers,post_id,post_shortcode,brand
0,1.794235e+16,2019-01-01 16:42:54,Así es un día para disfrutar sana mente y con...,"{'id': '8679040928', 'is_verified': False, 'pr...",1.0,[],1947257691818809835,BsGDI2_Adnr,offcorss
0,1.801764e+16,2019-01-02 16:45:57,Buenos días que precio tienen?,"{'id': '2252949863', 'is_verified': False, 'pr...",1.0,"[{'id': 17953738438218094, 'created_at': 15464...",1947952244502703605,BsIhD7lAi31,offcorss
0,1.791861e+16,2019-01-02 18:48:08,🎊,"{'id': '9918008061', 'is_verified': False, 'pr...",1.0,[],1948073194162672935,BsI8j-uhtUn,offcorss
0,1.798609e+16,2019-01-03 03:33:23,Hermosa mi sheshe,"{'id': '4903013997', 'is_verified': False, 'pr...",1.0,[],1948284380053077836,BsJslI6gX9M,offcorss
1,1.790035e+16,2019-01-03 01:00:38,Cuanto cuesta el vestido de manga larga talla 14,"{'id': '2158477951', 'is_verified': False, 'pr...",3.0,"[{'id': 18020433541055266, 'created_at': 15464...",1948284380053077836,BsJslI6gX9M,offcorss


-----------------------------


## Cleaning the posts DF
First, we are going to change so that they correspond to the same names of the database design.

In [6]:
ig_posts.reset_index(drop=True, inplace=True)

ig_posts.rename(columns={
    'id':'post_id',
    'edge_media_to_caption':'text',
    'taken_at_timestamp':'time',
    '__typename':'type',
    'edge_media_preview_like':'total_responses',
    'edge_media_to_comment':'total_comments',
    'posts_company':'brand_username'
}, inplace=True)

ig_posts.head()

,post_id,shortcode,text,time,type,is_ad,is_video,total_responses,total_comments,brand_username
0,1946502669929734835,BsDXd2DguKz,{'edges': [{'node': {'text': '¡FELIZ AÑO NUEVO...,1546261222,GraphImage,NaN,False,{'count': 308},{'count': 0},offcorss
1,1946624133611601575,BsDzFX7Hz6n,{'edges': [{'node': {'text': '¡Hoy es un día p...,1546275701,GraphSidecar,NaN,False,{'count': 212},{'count': 0},offcorss
2,1946744414957729955,BsEObsqB_ij,{'edges': [{'node': {'text': '¡Estamos listos ...,1546290040,GraphImage,NaN,False,{'count': 278},{'count': 0},offcorss
3,1947257691818809835,BsGDI2_Adnr,{'edges': [{'node': {'text': 'Día 1 del año 🎉⭐...,1546351227,GraphImage,False,False,"{'count': 126, 'edges': []}","{'count': 1, 'edges': [], 'page_info': {'end_c...",offcorss
4,1947346707928591260,BsGXYNsnCuc,{'edges': [{'node': {'text': 'UN NUEVO AÑO com...,1546361839,GraphSidecar,NaN,False,{'count': 391},{'count': 0},offcorss


Now, we see that the columns `text`, `total_responses` and `total_comments` have a dictionary in them, but we are only interested in one field. Then, we are going to extract that value of those columns.

In [7]:
ig_posts['text'] = ig_posts.text.apply(lambda x: x['edges'][0]['node']['text'] if x['edges'] != [] else "")
ig_posts['total_responses'] = ig_posts.total_responses.apply(lambda x: x['count'])
ig_posts['total_comments'] = ig_posts.total_comments.apply(lambda x: x['count'])

ig_posts.head()

,post_id,shortcode,text,time,type,is_ad,is_video,total_responses,total_comments,brand_username
0,1946502669929734835,BsDXd2DguKz,¡FELIZ AÑO NUEVO para todos! 😎✨❤ Les deseamos ...,1546261222,GraphImage,NaN,False,308,0,offcorss
1,1946624133611601575,BsDzFX7Hz6n,¡Hoy es un día para AGRADECER por cada uno de ...,1546275701,GraphSidecar,NaN,False,212,0,offcorss
2,1946744414957729955,BsEObsqB_ij,¡Estamos listos para celebrar y recibir el 201...,1546290040,GraphImage,NaN,False,278,0,offcorss
3,1947257691818809835,BsGDI2_Adnr,Día 1 del año 🎉⭐🌈 Un día para disfrutar en fam...,1546351227,GraphImage,False,False,126,1,offcorss
4,1947346707928591260,BsGXYNsnCuc,"UN NUEVO AÑO comienza, y viene lleno de nuevos...",1546361839,GraphSidecar,NaN,False,391,0,offcorss


It should be interesting for the analysis to have the associed URL to the post. After exploring the instagram interface we have seen that all the post have the same URL (https://www.instagram.com/p/.../) where the `...` is an unique identificator of each post. Then, we have realized that the code used is the same in the `shortcode` column, so we created the `post_url` column with that information.

Also, we converted the `time` column to date format, and we extracted the hashtags from the `text` column.

In [8]:
ig_posts["time"] = ig_posts.time.apply(datetime.fromtimestamp)
ig_posts["post_url"] = ig_posts.shortcode.apply(lambda x: f'https://www.instagram.com/p/{x}/')
ig_posts["hashtags"] = ig_posts.text.apply(lambda t: [h.match for h in p.parse(t).hashtags] if p.parse(t).hashtags else None)

ig_posts.head()

,post_id,shortcode,text,time,type,is_ad,is_video,total_responses,total_comments,brand_username,post_url,hashtags
0,1946502669929734835,BsDXd2DguKz,¡FELIZ AÑO NUEVO para todos! 😎✨❤ Les deseamos ...,2018-12-31 13:00:22,GraphImage,NaN,False,308,0,offcorss,https://www.instagram.com/p/BsDXd2DguKz/,"[#HappyNewYear2019, #OFFCORSS]"
1,1946624133611601575,BsDzFX7Hz6n,¡Hoy es un día para AGRADECER por cada uno de ...,2018-12-31 17:01:41,GraphSidecar,NaN,False,212,0,offcorss,https://www.instagram.com/p/BsDzFX7Hz6n/,"[#HappyNewYear, #OFFCORSS]"
2,1946744414957729955,BsEObsqB_ij,¡Estamos listos para celebrar y recibir el 201...,2018-12-31 21:00:40,GraphImage,NaN,False,278,0,offcorss,https://www.instagram.com/p/BsEObsqB_ij/,"[#HappyNewYear2019, #OFFCORSS]"
3,1947257691818809835,BsGDI2_Adnr,Día 1 del año 🎉⭐🌈 Un día para disfrutar en fam...,2019-01-01 14:00:27,GraphImage,False,False,126,1,offcorss,https://www.instagram.com/p/BsGDI2_Adnr/,"[#FelizAñoNuevo, #OFFCORSS]"
4,1947346707928591260,BsGXYNsnCuc,"UN NUEVO AÑO comienza, y viene lleno de nuevos...",2019-01-01 16:57:19,GraphSidecar,NaN,False,391,0,offcorss,https://www.instagram.com/p/BsGXYNsnCuc/,"[#FelizAñoNuevo, #OFFCORSS]"


---------------------------

## Cleaning the comments DF

In [9]:
ig_comments.head()

,id,created_at,text,owner,likes_count,answers,post_id,post_shortcode,brand
0,1.794235e+16,2019-01-01 16:42:54,Así es un día para disfrutar sana mente y con...,"{'id': '8679040928', 'is_verified': False, 'pr...",1.0,[],1947257691818809835,BsGDI2_Adnr,offcorss
0,1.801764e+16,2019-01-02 16:45:57,Buenos días que precio tienen?,"{'id': '2252949863', 'is_verified': False, 'pr...",1.0,"[{'id': 17953738438218094, 'created_at': 15464...",1947952244502703605,BsIhD7lAi31,offcorss
0,1.791861e+16,2019-01-02 18:48:08,🎊,"{'id': '9918008061', 'is_verified': False, 'pr...",1.0,[],1948073194162672935,BsI8j-uhtUn,offcorss
0,1.798609e+16,2019-01-03 03:33:23,Hermosa mi sheshe,"{'id': '4903013997', 'is_verified': False, 'pr...",1.0,[],1948284380053077836,BsJslI6gX9M,offcorss
1,1.790035e+16,2019-01-03 01:00:38,Cuanto cuesta el vestido de manga larga talla 14,"{'id': '2158477951', 'is_verified': False, 'pr...",3.0,"[{'id': 18020433541055266, 'created_at': 15464...",1948284380053077836,BsJslI6gX9M,offcorss


In [10]:
ig_comments.reset_index(drop=True, inplace=True)

ig_comments.rename(columns={
    "id":"response_id",
    "brand":"brand_username",
    "post_id":"rootPost_id",
    "owner":"username",
    "created_at":"time",
    "likes_count":"likes"
}, inplace=True)

ig_comments.head()

,response_id,time,text,username,likes,answers,rootPost_id,post_shortcode,brand_username
0,1.794235e+16,2019-01-01 16:42:54,Así es un día para disfrutar sana mente y con...,"{'id': '8679040928', 'is_verified': False, 'pr...",1.0,[],1947257691818809835,BsGDI2_Adnr,offcorss
1,1.801764e+16,2019-01-02 16:45:57,Buenos días que precio tienen?,"{'id': '2252949863', 'is_verified': False, 'pr...",1.0,"[{'id': 17953738438218094, 'created_at': 15464...",1947952244502703605,BsIhD7lAi31,offcorss
2,1.791861e+16,2019-01-02 18:48:08,🎊,"{'id': '9918008061', 'is_verified': False, 'pr...",1.0,[],1948073194162672935,BsI8j-uhtUn,offcorss
3,1.798609e+16,2019-01-03 03:33:23,Hermosa mi sheshe,"{'id': '4903013997', 'is_verified': False, 'pr...",1.0,[],1948284380053077836,BsJslI6gX9M,offcorss
4,1.790035e+16,2019-01-03 01:00:38,Cuanto cuesta el vestido de manga larga talla 14,"{'id': '2158477951', 'is_verified': False, 'pr...",3.0,"[{'id': 18020433541055266, 'created_at': 15464...",1948284380053077836,BsJslI6gX9M,offcorss


From this dataframe there isn´t too much to do. We are going to take only the `ùsername` for each dictionary of that column, and we are going to convert the `likes` column to int type.

In [11]:
ig_comments["likes"] = ig_comments.likes.astype('int')
ig_comments["username"] = ig_comments.username.apply(lambda x: x['username'])

ig_comments.head()

,response_id,time,text,username,likes,answers,rootPost_id,post_shortcode,brand_username
0,1.794235e+16,2019-01-01 16:42:54,Así es un día para disfrutar sana mente y con...,rosaliafaneite,1,[],1947257691818809835,BsGDI2_Adnr,offcorss
1,1.801764e+16,2019-01-02 16:45:57,Buenos días que precio tienen?,aleja.vasquez911,1,"[{'id': 17953738438218094, 'created_at': 15464...",1947952244502703605,BsIhD7lAi31,offcorss
2,1.791861e+16,2019-01-02 18:48:08,🎊,pactrii188,1,[],1948073194162672935,BsI8j-uhtUn,offcorss
3,1.798609e+16,2019-01-03 03:33:23,Hermosa mi sheshe,dalekeyboutique,1,[],1948284380053077836,BsJslI6gX9M,offcorss
4,1.790035e+16,2019-01-03 01:00:38,Cuanto cuesta el vestido de manga larga talla 14,delgadoa953,3,"[{'id': 18020433541055266, 'created_at': 15464...",1948284380053077836,BsJslI6gX9M,offcorss


As we can see, there are all the comments for all the scraped posts, but there is a column named `answers` that conteins all the answers to that comment. From that column we are going to take the same fields that a normal comment.

In [12]:
ig_answers = ig_comments[ig_comments.answers.apply(len) != 0][['rootPost_id', 'response_id', 'answers', 'brand_username']].reset_index(drop=True)

ig_answers.head()

,rootPost_id,response_id,answers,brand_username
0,1947952244502703605,1.801764e+16,"[{'id': 17953738438218094, 'created_at': 15464...",offcorss
1,1948284380053077836,1.790035e+16,"[{'id': 18020433541055266, 'created_at': 15464...",offcorss
2,1948805488864773596,1.796902e+16,"[{'id': 18063949429025469, 'created_at': 15570...",offcorss
3,1948805488864773596,1.799455e+16,"[{'id': 17846744674342193, 'created_at': 15479...",offcorss
4,1948805488864773596,1.788860e+16,"[{'id': 17997127078139199, 'created_at': 15479...",offcorss


In [13]:
ig_answers = pd.DataFrame([{"rootPost_id":i[0], "parentPost_id":i[1], **j, "brand_username":i[3]} 
                           for i in ig_answers.to_numpy() for j in i[2]])

ig_answers.head()

,rootPost_id,parentPost_id,id,created_at,text,owner,likes_count,brand_username
0,1947952244502703605,1.801764e+16,17953738438218094,1546491596,Hola @aleja.vasquez911 la camiseta de laya la ...,"{'id': '222130816', 'is_verified': True, 'prof...",3,offcorss
1,1948284380053077836,1.790035e+16,18020433541055266,1546491691,Hola @delgadoa953 lo encuentras por $69.900. D...,"{'id': '222130816', 'is_verified': True, 'prof...",1,offcorss
2,1948805488864773596,1.796902e+16,18063949429025469,1557077648,@offcorss no es extraña para mí su respuesta.....,"{'id': '11526821956', 'is_verified': False, 'p...",0,offcorss
3,1948805488864773596,1.799455e+16,17846744674342193,1547948682,Hola @erika_vanessa_rios te enviaremos un enla...,"{'id': '222130816', 'is_verified': True, 'prof...",1,offcorss
4,1948805488864773596,1.788860e+16,17997127078139199,1547948705,Hola @jenny_paez te enviaremos un enlace para ...,"{'id': '222130816', 'is_verified': True, 'prof...",1,offcorss


In [14]:
ig_answers.rename(columns = {
    "id":"response_id",
    "created_at":"time",
    "owner":"username",
    "likes_count": "likes"
}, inplace=True)

ig_answers["time"] = ig_answers.time.apply(datetime.fromtimestamp)
ig_answers["username"] = ig_answers.username.apply(lambda x: x["username"])

ig_answers.head()

,rootPost_id,parentPost_id,response_id,time,text,username,likes,brand_username
0,1947952244502703605,1.801764e+16,17953738438218094,2019-01-03 04:59:56,Hola @aleja.vasquez911 la camiseta de laya la ...,offcorss,3,offcorss
1,1948284380053077836,1.790035e+16,18020433541055266,2019-01-03 05:01:31,Hola @delgadoa953 lo encuentras por $69.900. D...,offcorss,1,offcorss
2,1948805488864773596,1.796902e+16,18063949429025469,2019-05-05 17:34:08,@offcorss no es extraña para mí su respuesta.....,mapu0831,0,offcorss
3,1948805488864773596,1.799455e+16,17846744674342193,2019-01-20 01:44:42,Hola @erika_vanessa_rios te enviaremos un enla...,offcorss,1,offcorss
4,1948805488864773596,1.788860e+16,17997127078139199,2019-01-20 01:45:05,Hola @jenny_paez te enviaremos un enlace para ...,offcorss,1,offcorss


----------------------

## Generating the CSVs
Now, we are going to split the dataframe `ig_posts` into two CSVs corresponding to the `instagram_posts` and `instagram_reactions` tables. Also, we are going to combine the `ig_comments` and `ig_answers` dataframe into a CSV for the table `instagram_responses`.

In [15]:
instagram_posts = ig_posts[['post_id', 'text', 'time', 'type', 'is_ad', 'is_video', "post_url", "hashtags", 'brand_username']]
instagram_reactions = ig_posts[['post_id', 'brand_username', 'total_responses', 'total_comments']]

In [16]:
instagram_posts.to_csv(os.path.join('data', 'instagram_posts.csv'), encoding="utf-8-sig")
instagram_reactions.to_csv(os.path.join('data', 'instagram_reactions.csv'), encoding="utf-8-sig")

In [17]:
ig_comments["parentPost_id"] = ig_comments.response_id

instagram_responses = pd.concat([ig_answers, ig_comments[['rootPost_id', 'parentPost_id', 'response_id', 'time', 'text', 'username', 'likes', 'brand_username']]])

In [18]:
instagram_responses.to_csv(os.path.join('data', 'instagram_responses.csv'), encoding="utf-8-sig")